In [ ]:
#Importações 

from selenium import webdriver 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By 
from datetime import datetime
from PIL import Image
import pandas as pd
import pytesseract
import time
import os
import re


In [ ]:
# Inicializar o WebDriver do Selenium (neste caso, usando o Chrome)

service = Service()

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service, options=options)

In [ ]:
# Diretório onde os arquivos de texto estão localizados(mudar para o diretório onde se encontra este arquivo)
diretorio = r'SEU_DIRETORIO'


In [ ]:
# Url do site

url = 'https://rpachallengeocr.azurewebsites.net/'

In [ ]:
# Função para fazer o download por paginação

def fazer_download_links_pagina():
    
    janela_principal = driver.current_window_handle

    linhas_tabela = driver.find_elements(By.XPATH, '//tbody/tr')

    # Iterar sobre as linhas da tabela
    for linha in linhas_tabela:
        try:
            # Encontrar o link de download na linha atual
            link_download = linha.find_element(By.XPATH, './/td/a')
           
            # Obter o URL do link de download
            url_download = link_download.get_attribute('href')

            # Verificar o número de janela antes de abrir o link de download
            janelas_anteriores = set(driver.window_handles)

            link_download.click()

            # Esperar até que uma nova janela seja aberta
            WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(len(janelas_anteriores) + 1))
           
            # Verificar o número de janela antes de trocar de janela
            janelas_atuais = set(driver.window_handles)
            nova_alca_janela = janelas_atuais - janelas_anteriores
            
            # Trocar de jhanela
            driver.switch_to.window(nova_alca_janela.pop())

            # Esperar até que a nova aba seja completamente carregada e então voltar para a janela principal
            WebDriverWait(driver, 10).until(EC.url_to_be(url_download))
            driver.switch_to.window(janela_principal)

        # Se ocorrer a exceção StaleElementReferenceException, continuamos para a próxima iteração
        except StaleElementReferenceException:
            continue

In [ ]:
# Criar listas e armazena os valores extraídos

def cria_arquivo_tabelas():
    
    ids = []
    due_dates = []
    linhas_tabela = driver.find_elements(By.XPATH, '//tbody/tr')

    # Iterar sobre as linhas da tabela
    for linha in linhas_tabela:
        id = linha.find_element(By.XPATH, './td[2]').text
        due_date = linha.find_element(By.XPATH, './td[3]').text
        
        ids.append(id)
        due_dates.append(due_date)

    df = pd.DataFrame({'ID': ids, 'Due Date': due_dates})

    print(df)

    # Verificar se o arquivo CSV já existe
    if os.path.exists('dados_tabela.csv'):
        # Se o arquivo existir, anexar os novos dados ao arquivo
        df.to_csv('dados_tabela.csv', mode='a', header=False, index=False)
        print("Novos dados anexados ao arquivo 'dados_tabela.csv'.")
    else:
        # Se o arquivo não existir, salvar o DataFrame em um novo arquivo CSV
        df.to_csv('dados_tabela.csv', index=False)
        print("Arquivo 'dados_tabela.csv' criado com sucesso.")

In [ ]:
 # Abrir a página inicial

driver.get(url)

In [ ]:

# Loop para navegar por todas as páginas da tabela

while True:
    cria_arquivo_tabelas()
    fazer_download_links_pagina()

    # verifica se o botão "Next" está desabilitado
    next_button = driver.find_element(By.ID, "tableSandbox_next")
    if "disabled" in next_button.get_attribute("class"):
        break

    next_button.click()

    time.sleep(3)

In [ ]:
# Obter todas as janelas do navegador
janelas = driver.window_handles

# Se houver mais de uma janela, começar da segunda
if len(janelas) > 1:
    # Iterar sobre as janelas, começando da segunda
    for idx, janela in enumerate(janelas[1:], start=1):
        # Mudar para a janela atual
        driver.switch_to.window(janela)

        # Baixar a imagem
        nome_arquivo_imagem = f"imagem_{idx}.png"
        driver.save_screenshot(nome_arquivo_imagem)

        # Extrair o texto da imagem
        imagem = Image.open(nome_arquivo_imagem)
        texto_extraido = pytesseract.image_to_string(imagem)

        # Salvar o texto em um arquivo de texto
        nome_arquivo_texto = f"texto_{idx}.txt"
        with open(nome_arquivo_texto, 'w') as arquivo_texto:
            arquivo_texto.write(texto_extraido)

        # Remover a imagem baixada
        os.remove(nome_arquivo_imagem)


In [ ]:
# Função para extrair informações de um arquivo de texto
def extrair_informacoes_arquivo(nome_arquivo):
    with open(nome_arquivo, 'r') as arquivo_texto:
        texto = arquivo_texto.read()

    # Padrões de regex para extrair as informações desejadas
    padrao_invoice_number = r'#(\d+)'
    padrao_invoice_date = r'(\w{3}\s+\d{1,2},\s+\d{4}|\d{4}-\d{2}-\d{2})'
    padrao_company_name = r'(Aenean\s*LLC|Sit\s*Amet\s*Corp)'
    padrao_total_due = r'(?:Balance\s*Due\s*:\s*\$?|Total\s*)([\d,.]+)'

    # Extrair as informações usando regex
    invoice_number_match = re.search(padrao_invoice_number, texto)
    invoice_date_match = re.search(padrao_invoice_date, texto)
    company_name_match = re.search(padrao_company_name, texto)
    total_due_match = re.search(padrao_total_due, texto)

    # Armazenar as informações extraídas
    informacoes = {}

    # Verificar se as informações foram encontradas e adicionar ao dicionário
    if invoice_number_match:
        informacoes['Invoice Number'] = invoice_number_match.group(1)
    if invoice_date_match:
        # Verificar o formato da data
        data = invoice_date_match.group(1)
        if re.match(r'\d{4}-\d{2}-\d{2}', data):
            # Se a data estiver no formato 'yyyy-mm-dd', converter para 'dd-mm-yyyy'
            data_formatada = datetime.strptime(data, '%Y-%m-%d').strftime('%d-%m-%Y')
            informacoes['Invoice Date'] = data_formatada
        elif re.match(r'\w{3}\s+\d{1,2},\s+\d{4}', data):
            # Se a data estiver no formato 'Jun 1, 2019', converter para 'dd-mm-yyyy'
            data_formatada = datetime.strptime(data, '%b %d, %Y').strftime('%d-%m-%Y')
            informacoes['Invoice Date'] = data_formatada
    if company_name_match:
        informacoes['Company Name'] = company_name_match.group(1)
    if total_due_match:
        informacoes['Total Due'] = total_due_match.group(1)

    # Retornar as informações extraídas
    return informacoes

# Lista para armazenar as informações extraídas de cada arquivo
informacoes_extraidas = []

# Iterar sobre os arquivos de texto no diretório
for arquivo in os.listdir(diretorio):
    if arquivo.endswith('.txt'):
        caminho_arquivo = os.path.join(diretorio, arquivo)
        informacoes = extrair_informacoes_arquivo(caminho_arquivo)
        if informacoes:
            # Adicionar as informações extraídas à lista
            informacoes_extraidas.append(informacoes)
        else:
            print(f"Falha ao extrair informações do arquivo {arquivo}")

# Criar um DataFrame com as informações extraídas
df = pd.DataFrame(informacoes_extraidas)

# Salvar o DataFrame em um arquivo CSV
df.to_csv('informacoes_extraidas.csv', index=False)

print("Informações extraídas:")
print(df)


In [ ]:
# Exclui os arquvios de texto desnecessarios
for arquivo in os.listdir(diretorio):
    if arquivo.endswith('.txt'):
        caminho_arquivo = os.path.join(diretorio, arquivo)
        os.remove(caminho_arquivo)
        print(f"Arquivo {arquivo} removido com sucesso.")


In [ ]:
# Carregar os DataFrames
dados_tabela = pd.read_csv('dados_tabela.csv')
informacoes_extraidas = pd.read_csv('informacoes_extraidas.csv')

# Mesclar os DataFrames com base no índice
tabela_completa = pd.concat([informacoes_extraidas, dados_tabela], axis=1)

# Salvar o DataFrame resultante em um novo arquivo CSV
tabela_completa.to_csv('tabela_completa.csv', index=False)

# Exibir o DataFrame resultante
print("Tabela completa:")
print(tabela_completa)


In [ ]:
# Carregar a tabela
tabela_completa = pd.read_csv('tabela_completa.csv')

data_atual = pd.to_datetime(datetime.now().date())

# Converter a coluna 'due date' para o tipo datetime
tabela_completa['Due Date'] = pd.to_datetime(tabela_completa['Due Date'], format='%d-%m-%Y')

# Manter apenas as os Invoices que já venceram
tabela_completa = tabela_completa[tabela_completa['Due Date'] <= data_atual]
tabela_completa.to_csv('tabela_completa_atualizada.csv', index=False)

print("Tabela completa atualizada:")
print(tabela_completa)


In [ ]:
# Listar todos os arquivos no diretório
arquivos = os.listdir(diretorio)

# Iterar sobre os arquivos
for arquivo in arquivos:
    # Verificar se é um arquivo CSV e não é 'tabela_completa_atualizada.csv' e exclui o resto
    if arquivo.endswith('.csv') and arquivo != 'tabela_completa_atualizada.csv':
        caminho_arquivo = os.path.join(diretorio, arquivo)
        os.remove(caminho_arquivo)

print("Arquivos CSV excluídos com sucesso, exceto 'tabela_completa_atualizada.csv'.")

# Encerra a instância do navegador

driver.quit()